# # Table of Contents
# 1. [Importing Libraries](#import-libraries)
# 2. [Hyperparam Tuning Optuna](#hyperparam-tuning-optuna)

# # Importing Libraries <a id="import-libraries"></a>

In [1]:
import optuna
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error

c:\Users\Usuario\.virtualenvs\predictive_salary_model-EQFagWwt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('C:/Users/Usuario/Documents/prueba_pwc/predictive_salary_model/data/processed/dataset_features.csv')

# # Hyperparam Tuning Optuna <a id="hyperparam-tuning-optuna"></a>

In [25]:
def objective(trial):

    n_estimators = trial.suggest_int("n_estimators", 50, 300, step=50)
    max_depth = trial.suggest_categorical("max_depth", [None, 10, 20, 30])
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    rf = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
        n_jobs=-1
    )

    scores = cross_val_score(
        rf,
        X, 
        y,
        scoring="neg_mean_absolute_error",
        cv=5,
        n_jobs=-1
    )
    mae = -np.mean(scores) 
    return mae

def run_optuna_search(df, n_trials=20):

    global X, y 
  
    X = df.drop(columns=["Salary"], errors="ignore")
    y = df["Salary"]

    study = optuna.create_study(direction="minimize") 
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    print("Number of finished trials:", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print(f"  Value (MAE): {trial.value:.2f}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    best_params = trial.params
    rf_best = RandomForestRegressor(
        n_estimators=best_params["n_estimators"],
        max_depth=best_params["max_depth"],
        min_samples_split=best_params["min_samples_split"],
        random_state=42,
        n_jobs=-1
    )
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    rf_best.fit(X_train, y_train)
    preds = rf_best.predict(X_test)
    final_mae = mean_absolute_error(y_test, preds)
    print(f"MAE en test con hiperparámetros óptimos: {final_mae:.2f}")

In [26]:
run_optuna_search(df, n_trials=50)

[I 2025-01-19 12:58:18,918] A new study created in memory with name: no-name-b72a1e7f-2e8d-41df-bf19-9d51a21acfb8
Best trial: 0. Best value: 1359.75:   2%|▏         | 1/50 [00:00<00:21,  2.32it/s]

[I 2025-01-19 12:58:19,349] Trial 0 finished with value: 1359.7466593315216 and parameters: {'n_estimators': 300, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 6}. Best is trial 0 with value: 1359.7466593315216.


Best trial: 1. Best value: 1030.25:   4%|▍         | 2/50 [00:00<00:20,  2.36it/s]

[I 2025-01-19 12:58:19,769] Trial 1 finished with value: 1030.251314143125 and parameters: {'n_estimators': 300, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 4}. Best is trial 1 with value: 1030.251314143125.


Best trial: 1. Best value: 1030.25:   6%|▌         | 3/50 [00:01<00:18,  2.49it/s]

...

[I 2025-01-19 12:58:30,024] Trial 47 finished with value: 1047.2164027038707 and parameters: {'n_estimators': 150, 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 44 with value: 654.7927927927929.


Best trial: 44. Best value: 654.793:  98%|█████████▊| 49/50 [00:11<00:00,  4.35it/s]

[I 2025-01-19 12:58:30,234] Trial 48 finished with value: 905.0237889469856 and parameters: {'n_estimators': 100, 'max_depth': None, 'min_samples_split': 9, 'min_samples_leaf': 3}. Best is trial 44 with value: 654.7927927927929.


Best trial: 44. Best value: 654.793: 100%|██████████| 50/50 [00:11<00:00,  4.28it/s]

[I 2025-01-19 12:58:30,596] Trial 49 finished with value: 712.75747911748 and parameters: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 44 with value: 654.7927927927929.
Number of finished trials: 50
Best trial:
  Value (MAE): 654.79
  Params: 
    n_estimators: 100
    max_depth: None
    min_samples_split: 2
    min_samples_leaf: 1
MAE en test con hiperparámetros óptimos: 470.00
